# Import

In [49]:
import requests
import json
import pandas as pd
import os
import time


# Constants & Parameters

In [50]:
PHAVER_GRAPHQL_ENDPOINT = os.getenv("PHAVER_GRAPHQL_ENDPOINT")
PHAVER_PROFILE_ID = os.getenv("PHAVER_PROFILE_ID")

FIREBASE_TOKEN_URL = os.getenv("FIREBASE_API_URL") + os.getenv("FIREBASE_API_KEY")
FIREBASE_REFRESH_TOKEN = os.getenv("FIREBASE_REFRESH_TOKEN")

LIMIT_PER_REQUEST = 1000
MAX_FOLLOWINGS_REQUESTED = 5000

# Load GrapgQL Query

In [51]:
# Utility function to load a GraphQL query or fragment from a file
def load_graphql_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()
    
# Load fragments and query from their respective files
FRAGMENTS = load_graphql_file('graphql/fragments/Fragments.gql')
FOLLOWINGS_QUERY = load_graphql_file('graphql/queries/FollowingsQuery.gql')
FOLLOWERS_QUERY = load_graphql_file('graphql/queries/FollowersQuery.gql')
POINTS_QUERY = load_graphql_file('graphql/queries/PointsQuery.gql')
SET_FOLLOW_MUTATION = load_graphql_file('graphql/mutations/SetFollowMutation.gql')
PROFILE_QUERY = load_graphql_file('graphql/queries/ProfileQuery.gql')
QUOTA_QUERY = load_graphql_file('graphql/queries/QuotaQuery.gql')

# Functions

In [52]:
def request_access_token():
    payload = {
        "grantType": "refresh_token",
        "refreshToken": FIREBASE_REFRESH_TOKEN
    }

    response = requests.post(FIREBASE_TOKEN_URL, json=payload)
    response.raise_for_status()
    access_data = response.json()
    return access_data['access_token']

def phaver_graphql_api_request(query, variables, access_token):
    headers = {
        "Authorization": "Bearer " + access_token,
        "Content-Type": "application/json"
    }

    payload = {
        "query": query,
        "variables": variables
    }

    response = requests.post(
        PHAVER_GRAPHQL_ENDPOINT, 
        headers=headers, 
        json=payload
    )

    # Raise an exception for HTTP errors
    if response.status_code != 200:
        print(response.headers)
        response.raise_for_status()

    # Parse the JSON response
    data = response.json()

    # Check if the response contains any errors
    if 'errors' in data:
        raise Exception(data['errors'])
    
    time.sleep(1)
    return data

def addToClipBoard(text):
    command = 'echo ' + text.strip() + '| clip'
    os.system(command)

def save_to_json(data, filename):
    """Save the data to a JSON file"""
    # create the folder if it doesn't exist
    folder = os.path.dirname(filename)
    os.makedirs(folder, exist_ok=True)
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

# Request Token

In [53]:
access_token = request_access_token()
addToClipBoard("Bearer " + access_token)

# Followings Manager Functions

In [54]:
def request_followings(profile_id, limit_per_request, offset, access_token):
    """Fetch a batch of followings from the API"""

    data = phaver_graphql_api_request(
        query = FRAGMENTS + FOLLOWINGS_QUERY, 
        variables = {
            "profileId": profile_id,
            "limit": limit_per_request,
            "offset": offset
        }, 
        access_token = access_token
    )

    # get the followings from the response
    followings = data['data']['followings']
    
    return followings

def request_points(profile_id, access_token):
    """Fetch the points for a profile from the API"""

    data = phaver_graphql_api_request(
        query = FRAGMENTS + POINTS_QUERY, 
        variables = {
            "profileId": profile_id
        }, 
        access_token = access_token
    )
   
    # get the points from the response
    points = data['data']['phaverPoints']['phaverPointsCurrent']
    
    return points

def get_all_points(followings, access_token):
    """Fetch all points by iterating over paginated results"""
    for following in followings:
        profile_id = following['followedProfile']['id']
        points = request_points(profile_id, access_token)
        following['followedProfile']['points'] = points
    return followings


def get_all_followings(profile_id, limit_per_request, max_followings_requested, access_token):
    """Fetch all followings by iterating over paginated results"""
    all_followings = []
    offset = 0
    
    while len(all_followings) < max_followings_requested:
        followings = request_followings(profile_id, limit_per_request, offset, access_token)
        
        # If no more followings, break the loop
        if not followings:
            break
        
        # Append the fetched followings to the list
        all_followings.extend(followings)
        
        # Increment the offset by the limit for pagination
        offset += limit_per_request
        
    return all_followings

# Function to flatten the JSON data
def flatten_followings(followings):
    """Flatten the followings JSON data into Pandas DataFrame"""

    flattened_data = []

    for item in followings:
        followed_profile = item['followedProfile']
       
        # Flatten the nested fields
        flattened_profile = {
            "id": followed_profile.get("id"),
            "username": followed_profile.get("username"),
            "profileCreatedAt": followed_profile.get("createdAt"),
            "followingDate": item.get("createdAt"),
            "followerCount": followed_profile.get("profileAggregates", {}).get("followerCount") if followed_profile.get("profileAggregates") else None,
            "followingCount": followed_profile.get("profileAggregates", {}).get("followingCount") if followed_profile.get("profileAggregates") else None,
            "points": followed_profile.get("points"),
            "credLevel": followed_profile.get("credLevel"),
            "badge": followed_profile.get("badge"),
            "phaverFrens": followed_profile.get("phaverFrens"),
            "verification": followed_profile.get("verification"),
            "verified": followed_profile.get("verified"),
            "isUserFollowing": followed_profile.get("isUserFollowing"),
            "lensProfile.lensHandle": followed_profile.get("lensProfile", {}).get("lensHandle") if followed_profile.get("lensProfile") else None,
            "lensProfile.isUserFollowing": followed_profile.get("lensProfile", {}).get("isUserFollowing") if followed_profile.get("lensProfile") else None,
            "farcasterProfile.name": followed_profile.get("farcasterProfile", {}).get("name") if followed_profile.get("farcasterProfile") else None,
            "farcasterProfile.isUserFollowing": followed_profile.get("farcasterProfile", {}).get("isUserFollowing") if followed_profile.get("farcasterProfile") else None
        }

        # Append flattened profile to the list
        flattened_data.append(flattened_profile)

    return pd.DataFrame(flattened_data)

def request_profile(profile_id, access_token):
    """Fetch the profile data from the API"""

    data = phaver_graphql_api_request(
        query = FRAGMENTS + PROFILE_QUERY, 
        variables = {
            "profileId": profile_id
        }, 
        access_token = access_token
    )

    profile = data['data']['profile']
    return profile

def request_quota(account_id, access_token):
    """Fetch the quota data from the API"""

    data = phaver_graphql_api_request(
        query = FRAGMENTS + QUOTA_QUERY, 
        variables = {
            "accountId": account_id
        }, 
        access_token = access_token
    )

    quota = data['data']['quota']
    return quota

def unfollow_user(profile_id, network, access_token):
    if network not in ['phaver', 'lens', 'farcaster', 'all']:
        raise ValueError("Invalid network. Must be either 'phaver', 'lens', 'farcaster', or 'all'")
    if network == 'all':
        networks = ['phaver', 'lens', 'farcaster']
    else:
        networks = [network]

    profile = request_profile(profile_id, access_token)

    if profile is None:
        raise ValueError("Profile not found")
    
    # check if the user is not already unfollowing the profile on Phaver
    if 'phaver' in networks and not profile.get('isUserFollowing', False):
        networks.remove('phaver')

    # check if the profile has a Lens account, and if the user is not already unfollowing the profile on Lens
    if 'lens' in networks and (not profile.get('lensProfile') or not profile['lensProfile'].get('isUserFollowing', False)):
        networks.remove('lens')

    # check if the profile has a Farcaster account, and if the user is not already unfollowing the profile on Farcaster
    if 'farcaster' in networks and (not profile.get('farcasterProfile') or not profile['farcasterProfile'].get('isUserFollowing', False)):
        networks.remove('farcaster')

    if len(networks) == 0:
        return True

    for network in networks:
        data = phaver_graphql_api_request(
            query = FRAGMENTS + SET_FOLLOW_MUTATION, 
            variables = {
                "follow": False,
                "profileId": profile_id,
                "followType": network
            }, 
            access_token = access_token
        )

        if data['data']['setFollow']['status'] != 'ok':
            raise Exception(f"Failed to unfollow user on {network}")
    
    #Check that the profile is not being followed on any network
    profile = request_profile(profile_id, access_token)

    if profile.get('isUserFollowing', True):
        raise Exception("Failed to unfollow user on Phaver. Unkown error.\nprofile: " + json.dumps(profile, indent=4))
    if profile.get('lensProfile') and profile['lensProfile'].get('isUserFollowing', True):
        if not profile['lensProfile'].get('isFollowPending', False): # Lens takes some time to update the isUserFollowing field, so we need to check if the follow is pending. If it is, we can assume the unfollow was successful.
            quota = request_quota(PHAVER_PROFILE_ID, access_token)
            limit = min(quota['lensApiOnchain']['dailyAvailable'], quota['lensApiOnchain']['hourlyAvailable'])
            if limit <= 0:
                raise Exception("Failed to unfollow user on Lens. Onchain Lens API limit reached.\nprofile: " + json.dumps(profile, indent=4))
            else:
                raise Exception("Failed to unfollow user on Lens. Unkown error.\nprofile: " + json.dumps(profile, indent=4))
    if profile.get('farcasterProfile') and profile['farcasterProfile'].get('isUserFollowing', True):
        raise Exception("Failed to unfollow user on Farcaster. Unkown error.\nprofile: " + json.dumps(profile, indent=4))
        
    return True


# Request Followings

In [ ]:
# Fetch all followings for the given profile ID
followings = get_all_followings(PHAVER_PROFILE_ID, LIMIT_PER_REQUEST, MAX_FOLLOWINGS_REQUESTED, access_token)

# Get the points for each following
# followings = get_all_points(followings, access_token)

# Save the followings to followings.json
save_to_json(followings, 'data/followings.json')

print(f"Successfully saved {len(followings)} followings to 'data/followings.json'.")


# Convert JSON to DataFrame

In [64]:
# Flatten the data
followings_df = flatten_followings(followings)

# Add the current datetime to the DataFrame
followings_df['updatedAt'] = pd.Timestamp.now(tz='UTC')

# Add a tempo column to mark unfollowed users
followings_df['tempo_unfollowed'] = False

# Save the DataFrame to a CSV file
followings_df.to_csv('data/followings.csv', index=False)


# Load Saved Data

In [65]:
followings_df = pd.read_csv('data/followings.csv')

# Unfollow Profiles

In [ ]:
# Filter Profiles to Unfollow
profiles_to_unfollow_df = followings_df[
    (followings_df['followerCount'] < 2000) & 
    (followings_df['credLevel'] < 2) &
    (followings_df['badge'].isnull()) &
    (followings_df['verification'].isnull()) &
    # (followings_df['lensProfile.isUserFollowing'] != True) &
    (followings_df['tempo_unfollowed'] == False) 
]
print(f"Found {len(profiles_to_unfollow_df)} profiles to unfollow.")
profiles_to_unfollow_df

In [ ]:
try:
    # Unfollow Profiles
    for index, following in profiles_to_unfollow_df.iterrows():
        unfollowed = unfollow_user(following['id'], 'all', access_token)
        if unfollowed:
            print(f"Unfollowed {following['username']}")
            followings_df.loc[index, 'tempo_unfollowed'] = True
except Exception as e:
    # Save modified following list
    followings_df.to_csv('data/followings.csv', index=False)
    raise e
